In [1]:
import csv
import time
import torch
import torch.nn as nn
from torch.nn import functional as F
from contextlib import nullcontext

In [2]:
class CharLevelTokenizer:
    """Character-level tokenizer"""

    def __init__(self):
        self.char_to_id = {}
        self.id_to_char = {}
        self.vocab_size = 0

    def build_vocab(self, texts):
        """Build vocabulary from texts"""
        for text in sorted("".join(texts)):
            if text not in self.char_to_id:
                self.char_to_id[text] = self.vocab_size
                self.id_to_char[self.vocab_size] = text
                self.vocab_size += 1
        return self.vocab_size

    def encode(self, text):
        """Convert text to token IDs"""
        return [self.char_to_id[char] for char in text]

    def decode(self, ids):
        """Convert token IDs to text"""
        return "".join(self.id_to_char[token_id] for token_id in ids)

In [3]:
class Head(nn.Module):
    """One head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B, T, C)
        q = self.query(x)  # (B, T, C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [4]:
class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [5]:
class FeedFoward(nn.Module):
    """Simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [6]:
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [7]:
class GPTModel(nn.Module):
    """GPT model"""

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx):
        B, T = idx.shape

        # idx is (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B, T, C)
        pos_emb = self.position_embedding_table( torch.arange(T, device=device))  # (T, C)
        x = tok_emb + pos_emb  # (B, T, C)
        x = self.blocks(x)  # (B, T, C)
        x = self.ln_f(x)  # (B, T, C)
        logits = self.lm_head(x)  # (B, T, vocab_size)
        return logits

    def compute_loss(self, logits, targets):
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)
        return loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T + 1)
        return idx

In [8]:
def read_first_column(filepath, skip_header=True):
    """read csv dataset"""
    result = []
    with open(filepath, encoding="utf-8") as f:
        reader = csv.reader(f)
        if skip_header:
            next(reader)
        for row in reader:
            if row:
                col = row[0]
                if not col.upper().lstrip().startswith("AUDIENCE"):
                    result.append(col.partition(":")[2] or col.strip())
    return "\n".join(result)

In [9]:
def get_batch(split):
    """Data loading"""
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix]).to(device)
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix]).to(device)
    return x, y

In [10]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    results = {}
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits = model(X)
            loss = model.compute_loss(logits, Y)
            losses[k] = loss.item()
        results[split] = losses.mean()
    model.train()
    return results

In [11]:
batch_size = 128  # how many independent sequences will we process in parallel?
block_size = 256  # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 100
initial_lr = 3e-4
min_lr = 1e-4
eval_iters = 200
n_embd = 384
n_head = 3
n_layer = 3
dropout = 0.2

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
train_text = read_first_column("train.csv")
val_text = read_first_column("test.csv")

In [14]:
tokenizer = CharLevelTokenizer()
vocab_size = tokenizer.build_vocab(train_text)

In [15]:
torch.manual_seed(1337)
train_data = torch.tensor(tokenizer.encode(train_text), dtype=torch.long)
val_data = torch.tensor(tokenizer.encode(val_text), dtype=torch.long)

In [16]:
model = GPTModel()
print(f"{sum(p.numel() for p in model.parameters()) / 1e6} M parameters")

5.483604 M parameters


In [17]:
model = model.to(device)

In [18]:
optimizer = torch.optim.AdamW(model.parameters(), lr=initial_lr)
lambda_func = lambda epoch: max(0.99 ** epoch, min_lr / initial_lr)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_func)

In [19]:
dtype = 'float32' # 'float32' or 'bfloat16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16}[dtype]
ctx = nullcontext() if device == 'cpu' else torch.amp.autocast(device_type=device, dtype=ptdtype)
model = torch.compile(model)

In [20]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        t0 = time.time()
        losses = estimate_loss()
        t1 = time.time()
        dt = t1 - t0
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time {dt:.1f} secs")

    # sample a batch of data
    xb, yb = get_batch("train")

    with ctx:
        logits = model(xb)
    loss = model.compute_loss(logits, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

W0422 12:25:43.018000 664 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


step 0: train loss 4.4782, val loss 4.4783, time 59.5 secs
step 100: train loss 2.3640, val loss 2.3625, time 48.5 secs
step 200: train loss 2.2858, val loss 2.2879, time 48.3 secs
step 300: train loss 1.8112, val loss 1.8150, time 48.0 secs
step 400: train loss 1.5471, val loss 1.5508, time 48.0 secs
step 500: train loss 1.4154, val loss 1.4198, time 48.0 secs
step 600: train loss 1.3373, val loss 1.3449, time 47.9 secs
step 700: train loss 1.2834, val loss 1.2923, time 47.9 secs
step 800: train loss 1.2416, val loss 1.2529, time 48.0 secs
step 900: train loss 1.2066, val loss 1.2177, time 47.4 secs
step 999: train loss 1.1870, val loss 1.2010, time 46.8 secs


In [21]:
ctx = torch.tensor(tokenizer.encode("A"), dtype=torch.long).unsqueeze(0).to(device)
out = tokenizer.decode(model.generate(ctx, max_new_tokens=1000)[0].tolist())
print(out)

And Howere he Connell.
I countries it going to park it states the United States.
In the President oneas to aske Forces theses in West-Mal DFranahtra, States Prelss, Rerus systeday, no leaganization will tak and mystem find Auguss in administratione.
When you have people imme right.
I think that we will ide look be close is the President who made aleave into he schance to have accern and socantal Situgal of Ame wities. Secondle Bring lock. Heys notrow Department of the appoint that lack to it our pappointsmit to way and strength, come interestain.
The United States Administrationiter and controvernment this appening areapted overnovulved some oppmeonened the nations of border, I was anything that will do it.
And will lead that the Fed that competition address will falw.
Sememember with tis supportant seell you bime the has a vrick and derical light year and like it going to is the it to fight. We some ole backly with message in throus with and yeal I think commembat?
 Its talk about, th

In [22]:
!pip show torch

Name: torch
Version: 2.6.0+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision
